## Diabetic Retinopathy Detection

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow import keras

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
tf.config.list_physical_devices("GPU")

In [ ]:
print(tf.test.is_built_with_cuda())

In [ ]:
df = pd.read_csv(r"C:\Users\manic\Mini Project implementation\Data\train.csv")
df

In [ ]:
path= r"C:\Users\manic\Mini Project implementation\Data"

train_df , test_df = train_test_split(df,test_size=0.2, random_state=42)
train_df.to_csv(path+'\\train_s.csv',index=False)
test_df.to_csv(path+'\\test_s.csv',index=False)

In [ ]:
folder = r"C:\Users\manic\Mini Project implementation\Data\train_images"
train_image_paths = [folder + '\\' + img_id +'.png' for img_id in train_df['id_code'].values]
test_image_paths = [folder + '\\' + img_id + '.png' for img_id in test_df['id_code'].values]

In [ ]:
def train_func_image_file(x):
    folder = r"C:\Users\manic\Mini Project implementation\Data\train_images"
    path = folder + '\\' + x +'.png'
    return path

In [ ]:
train = pd.read_csv(r"C:\Users\manic\Mini Project implementation\Data\train_s.csv")
train['path'] = train['id_code'].apply(train_func_image_file)

In [ ]:
test = pd.read_csv(r"C:\Users\manic\Mini Project implementation\Data\test_s.csv")
test['path'] = test['id_code'].apply(train_func_image_file)

In [ ]:
train_labels = train['diagnosis'].values
test_labels = test['diagnosis'].values

In [ ]:
input_shape = (224,224,3)
num_classes = 5

In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(train['path'][i],target_size=input_shape,interpolation='nearest')
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

In [ ]:
x_train = np.array(train_image)

In [ ]:
y_train = train['diagnosis']
y_train = keras.utils.to_categorical(y_train,5)

In [ ]:
y_test = test['diagnosis']
y_test = keras.utils.to_categorical(y_test,5)

In [ ]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img(test['path'][i],target_size=input_shape,interpolation='nearest')
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
x_test = np.array(test_image)

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=3,validation_split = 0.2)
# Evaluate the model
accuracy = model.evaluate(x_test, y_test)[1]

## Hyperparameter tuning

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
def model_builder(hp):
    model_2 = Sequential()
    model_2.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model_2.add(MaxPooling2D((2, 2)))
    model_2.add(Conv2D(64, (3, 3), activation='relu'))
    model_2.add(MaxPooling2D((2, 2)))
    model_2.add(Conv2D(128, (3, 3), activation='relu'))
    model_2.add(MaxPooling2D((2, 2)))
    model_2.add(Flatten())
    model_2.add(Dense(128, activation='relu'))
    model_2.add(Dense(num_classes, activation='softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

# Compile the model
    model_2.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model_2

In [ ]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy',max_epochs=10,factor=3)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(x_train,y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.The optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
model_2 = tuner.hypermodel.build(best_hps)
history = model_2.fit(x_train,y_train, epochs=10, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
eval_result = model_2.evaluate(x_test,y_test)
print("[test loss, test accuracy]:", eval_result)